**Upper Bound (QP) - 2D**

In [1]:
import cvxpy as cp
import numpy as np
from typing import List

def g1_ub_qp_2D(seen: List[List[int]], row_sum: List[int], col_sum: List[int]) -> int:
    n = len(row_sum)
    m = len(col_sum)
    seen_array = np.array(seen)

    # optimization variables
    matrix = cp.Variable((n, m), nonneg=True)

    # precomputed row and column totals of the seen matrix
    seen_row_sums = seen_array.sum(axis=1)
    seen_col_sums = seen_array.sum(axis=0)

    # objective: minimize sum of squares of seen + matrix
    objective = cp.Minimize(cp.sum_squares(seen_array + matrix))

    # constraints for row and column sums
    constraints = [
        cp.sum(matrix, axis=1) + seen_row_sums <= row_sum,
        cp.sum(matrix, axis=0) + seen_col_sums <= col_sum
    ]

    # define solver
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.SCS)

    if problem.status != cp.OPTIMAL:
        raise ValueError("Optimization was not successful")

    g1ErrorUB = problem.value
    row_square = np.dot(row_sum, row_sum)
    conflict = (row_square - g1ErrorUB) / 2
    return conflict

**Lower Bound (Ignore Row Sum) - 2D**

In [2]:
import numpy as np
from typing import List

def g1_lb_ignoreSum_2D(seen: List[List[int]], row_sum: List[int], col_sum: List[int]) -> int:
  row_sum_cur = []
  row_max_index = []
  remaining_row_sum = []

  n = len(row_sum)
  m = len(col_sum)

  # Calculate conflict
  xi_square = 0
  for i in range(n):
    row_sum_cur.append(0)
    row_max_index.append(0)
    remaining_row_sum.append(0)
    for j in range(m):
      row_sum_cur[i] += seen[i][j]
      if seen[i][row_max_index[i]] < seen[i][j]:
        row_max_index[i] = j
      xi_square += seen[i][j]*seen[i][j]
    remaining_row_sum[i] = row_sum[i] - row_sum_cur[i]
    _max = seen[i][row_max_index[i]]
    xi_square += remaining_row_sum[i] * remaining_row_sum[i] + 2 * _max * remaining_row_sum[i]
  row_square = 0
  for r_s in row_sum:
    row_square += r_s*r_s
  conflict = (row_square - xi_square)/2
  return conflict

**Bounds (QP) - 3D**

In [3]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 86.6 MB/s eta 0:00:00


In [4]:
import gurobipy as gp
import numpy as np
from typing import List

def g1_bounds_qp_3D(
    seen: List[List[List[int]]],
    row_sum_1: List[int],
    row_sum_2: List[int],
    col_sum: List[int],
    lb_calc: bool
) -> int:

    n1 = len(row_sum_1)
    n2 = len(row_sum_2)
    m = len(col_sum)

    seen_np = np.array(seen)

    model = gp.Model()
    model.setParam("OutputFlag", 0)

    # non-negative decision variables
    matrix_vars = model.addVars(n1, n2, m, lb=0, name="matrix")

    # objective function: sum over (seen + x)[i,j,k1] * (seen + x)[i,j,k2] for k1 < k2
    objective_terms = []
    for i in range(n1):
        for j in range(n2):
            for k1 in range(m - 1):
                for k2 in range(k1 + 1, m):
                    t1 = seen_np[i, j, k1] + matrix_vars[i, j, k1]
                    t2 = seen_np[i, j, k2] + matrix_vars[i, j, k2]
                    objective_terms.append(t1 * t2)

    model.setObjective(gp.quicksum(objective_terms), gp.GRB.MINIMIZE if lb_calc else gp.GRB.MAXIMIZE)

    # row sum constraints
    for i in range(n1):
        rhs = row_sum_1[i] - seen_np[i, :, :].sum()
        model.addConstr(
            gp.quicksum(matrix_vars[i, j, k] for j in range(n2) for k in range(m)) <= rhs,
            name=f"RowSum1_{i}"
        )

    for j in range(n2):
        rhs = row_sum_2[j] - seen_np[:, j, :].sum()
        model.addConstr(
            gp.quicksum(matrix_vars[i, j, k] for i in range(n1) for k in range(m)) <= rhs,
            name=f"RowSum2_{j}"
        )

    # col sum constraints
    for k in range(m):
        rhs = col_sum[k] - seen_np[:, :, k].sum()
        model.addConstr(
            gp.quicksum(matrix_vars[i, j, k] for i in range(n1) for j in range(n2)) <= rhs,
            name=f"ColSum_{k}"
        )

    model.optimize()

    if model.status == gp.GRB.OPTIMAL:
        return model.objVal
    else:
        raise ValueError("No optimal solution found.")

**Bounds (QP) - 4D**

In [5]:
import gurobipy as gp
import numpy as np
from typing import List

def g1_bounds_qp_4D(
    seen: List[List[List[List[int]]]],
    row_sum_1: List[int],
    row_sum_2: List[int],
    row_sum_3: List[int],
    col_sum: List[int],
    lb_calc: bool
) -> int:

    n1 = len(row_sum_1)
    n2 = len(row_sum_2)
    n3 = len(row_sum_3)
    m = len(col_sum)

    seen_np = np.array(seen)

    model = gp.Model()
    model.setParam("OutputFlag", 0)

    # decision variables: non-negative
    matrix_vars = model.addVars(n1, n2, n3, m, lb=0, name="matrix")

    # objective: sum over (seen + matrix)[i,j,k,l1] * (seen + matrix)[i,j,k,l2] for l1 < l2
    objective_terms = []
    for i in range(n1):
        for j in range(n2):
            for k in range(n3):
                for l1 in range(m - 1):
                    for l2 in range(l1 + 1, m):
                        term1 = seen_np[i, j, k, l1] + matrix_vars[i, j, k, l1]
                        term2 = seen_np[i, j, k, l2] + matrix_vars[i, j, k, l2]
                        objective_terms.append(term1 * term2)

    if lb_calc:
        model.setObjective(gp.quicksum(objective_terms), gp.GRB.MINIMIZE)
    else:
        model.setObjective(gp.quicksum(objective_terms), gp.GRB.MAXIMIZE)

    # Constraints

    # 1D row sums
    for i in range(n1):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for j in range(n2) for k in range(n3) for l in range(m))
        rhs = row_sum_1[i] - seen_np[i, :, :, :].sum()
        model.addConstr(lhs <= rhs, name=f"RowSum1_{i}")

    # 2D row sums
    for j in range(n2):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for i in range(n1) for k in range(n3) for l in range(m))
        rhs = row_sum_2[j] - seen_np[:, j, :, :].sum()
        model.addConstr(lhs <= rhs, name=f"RowSum2_{j}")

    # 3D row sums
    for k in range(n3):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for i in range(n1) for j in range(n2) for l in range(m))
        rhs = row_sum_3[k] - seen_np[:, :, k, :].sum()
        model.addConstr(lhs <= rhs, name=f"RowSum3_{k}")

    # Column sums
    for l in range(m):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for i in range(n1) for j in range(n2) for k in range(n3))
        rhs = col_sum[l] - seen_np[:, :, :, l].sum()
        model.addConstr(lhs <= rhs, name=f"ColSum_{l}")

    model.optimize()

    if model.status == gp.GRB.OPTIMAL:
        return model.objVal
    else:
        raise ValueError("No optimal solution found.")

**Approx Top-K Ranking (Sampling/Mean)**

In [6]:
import random
from collections import defaultdict

# Approach 1 : Using mean to find top-k
def cal_mean(attributes, ubDict, lbDict, e):

  meanTopk = []
  emax_attr = []

  for attribute in attributes:
    lowerB = lbDict[attribute]
    upperB = ubDict[attribute]

    if upperB <= e:
        emax_attr.append(attribute)

  for attr in emax_attr:
    lb = lbDict[attr]
    ub = ubDict[attr]
    meanVal = (lb+ub)/2
    meanTopk.append((attr, meanVal))

  meanTopk = sorted(meanTopk, key=lambda x: x[1])

  meanTopk = [attr[0] for attr in meanTopk]

  return meanTopk

# Approach 2 : Using sampling to find top-k
def cal_sample_prob(attributes, ubDict, lbDict, e):

  emax_attr = []
  order_counts = defaultdict(int)

  for attribute in attributes:
    lowerB = lbDict[attribute]
    upperB = ubDict[attribute]

    if upperB <= e:
      emax_attr.append(attribute)

  for _ in range(1000):
    samples = {}
    for attr in emax_attr:
      lower_bound = lbDict[attr]
      upper_bound = ubDict[attr]
      samples[str(attr)] = random.uniform(lower_bound, upper_bound)

    sorted_order = tuple(sorted(samples, key=samples.get))
    order_counts[sorted_order] += 1

  most_frequent_order = max(order_counts, key=order_counts.get)

  return most_frequent_order

**Actual Top-K**

In [7]:
import numpy as np
import pandas as pd
import itertools
from itertools import combinations

def calculateActualTopk(file_name, e):

  mainDF = pd.read_csv(file_name, header = 0)
  # last column in dataset is the Z attribute
  A_name_list = mainDF.columns[:-1].tolist()

  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows-1)

  # THREE-LHS AFDs

  actualConflictDict_4D = {}

  attribute_triplets = list(combinations(mainDF.columns[:-1], 3))

  for attr1, attr2, attr3 in attribute_triplets:

    A_name = attr1+attr2+attr3

    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    attr3_values = mainDF[attr3].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_3lhs = [[[[0 for _ in target_values] for _ in attr3_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    attr3_indices_map = {value: idx for idx, value in enumerate(attr3_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    rowSum_a3 = [0]*len(attr3_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      a3_index = attr3_indices_map[row[attr3]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      rowSum_a3[a3_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      val1 = row[attr1]
      val2 = row[attr2]
      val3 = row[attr3]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and val3 in attr3_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        depth_index = attr3_indices_map[val3]
        target_index = target_indices_map[target_val]

        matrix_3lhs[row_index][col_index][depth_index][target_index] += 1

    g1Error_4D = 0

    for matrix_3d in matrix_3lhs:
      for matrix_2d in matrix_3d:
          for row in matrix_2d:
              pairs = itertools.combinations(row, 2)
              for pair in pairs:
                  g1Error_4D += pair[0] * pair[1]

    actualConflictDict_4D[str(A_name)] = g1Error_4D/norm

  # THREE-LHS AFDs

  # TWO-LHS AFDs

  actualConflictDict_3D = {}

  attribute_pairs = list(combinations(mainDF.columns[:-1], 2))

  for attr1, attr2 in attribute_pairs:

    A_name = attr1+attr2
    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_2lhs = [[[0 for _ in target_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      val1 = row[attr1]
      val2 = row[attr2]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        target_index = target_indices_map[target_val]

        matrix_2lhs[row_index][col_index][target_index] += 1

    g1Error_3D = 0

    for matrix_2d in matrix_2lhs:
      for row in matrix_2d:
          pairs = itertools.combinations(row, 2)
          for pair in pairs:
              g1Error_3D += pair[0] * pair[1]

    actualConflictDict_3D[str(A_name)] = g1Error_3D/norm

  # TWO-LHS AFDs

  # ONE-LHS AFDs

  actualConflictDict_2D = {}

  for A_name in A_name_list:

    Z_name = "y"
    df = mainDF[[A_name, Z_name]]

    a_indices = np.unique(df[A_name])
    a_indices = a_indices.tolist()
    a_indices_map = dict(zip(a_indices, range(len(a_indices))))

    z_indices = np.unique(df[Z_name])
    z_indices = z_indices.tolist()
    z_indices_map = dict(zip(z_indices, range(len(z_indices))))

    row_sum = [0]*len(a_indices)
    col_sum = [0]*len(z_indices)

    matrix = []
    for i in a_indices:
      matrix.append([0]*len(z_indices))

    dataset = df.values.tolist()

    for data in dataset:
      row_index = a_indices_map[data[0]]
      col_index = z_indices_map[data[1]]
      row_sum[row_index] += 1
      col_sum[col_index] += 1
      matrix[row_index][col_index] += 1

    g1Error_2D = 0
    for row in matrix:
      pairs = itertools.combinations(row, 2)
      for pair in pairs:
        g1Error_2D += pair[0] * pair[1]

    actualConflictDict_2D[A_name] = g1Error_2D/norm

  # ONE-LHS AFDs

  mergedConflictDict = {**actualConflictDict_2D, **actualConflictDict_3D, **actualConflictDict_4D}
  unsortedConflictDict = {k: v for k, v in mergedConflictDict.items() if v <= e}
  sortedConflictDict = {k: v for k, v in sorted(mergedConflictDict.items(), key=lambda item: item[1]) if v <= e}
  actualTopk = sorted(sortedConflictDict, key=lambda k: sortedConflictDict[k])

  return sortedConflictDict, unsortedConflictDict

**LPDS - Uninformed Prior**

In [8]:
import os
import time
import numpy as np
import pandas as pd

def removeSingleMarginals(calc_matrix, row_sum, col_sum):

  row_sum_copy = row_sum[:]
  col_sum_copy = col_sum[:]
  matrix_copy = [row[:] for row in calc_matrix]

  rows_to_remove = [i for i, rsum in enumerate(row_sum_copy) if rsum == 1]

  for i in sorted(rows_to_remove, reverse=True):
        row = matrix_copy[i]
        for j, val in enumerate(row):
            col_sum_copy[j] -= val

        del matrix_copy[i]
        del row_sum_copy[i]

  return matrix_copy, row_sum_copy, col_sum_copy

def calculateBoundsLPDS(file_name, dataInterval, stopAtIndex, e):

  attrPrunedLP = []
  banned_attributes = set()
  mainUBDict, mainLBDict, mainLPDict = {}, {}, {}

  mainDF = pd.read_csv(file_name, header=0)
  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows - 1)
  A_name_list = mainDF.columns[:-1].tolist()

  currentC = [[a] for a in A_name_list]

  while currentC:

    nextC = []
    seen = set()

    for lhs in currentC:

      if len(lhs) > 3:
        continue # upto l=3
      if any(attr in banned_attributes for attr in lhs):
        continue  # prune early if attr banned

      A_name = ''.join(lhs)
      Z_name = "y"
      df = mainDF[lhs + [Z_name]]

      # mapping attr values and constructing matrix, rowsum, colsum
      attr_maps = [{val: i for i, val in enumerate(df[col].unique())} for col in lhs]
      z_values = df[Z_name].unique().tolist()
      z_map = {val: i for i, val in enumerate(z_values)}
      shape = [len(m) for m in attr_maps] + [len(z_values)]

      matrix = np.zeros(shape, dtype=int)
      col_sum = [0] * len(z_values)
      row_sums = [[0] * s for s in shape[:-1]]

      u_final = 1.0
      l_final = 0.0

      for idx, row in enumerate(df.itertuples(index=False)):

        if idx > stopAtIndex:
          break

        coords = [attr_maps[i][row[i]] for i in range(len(lhs))]
        z_idx = z_map[row[-1]]
        matrix[tuple(coords + [z_idx])] += 1
        for i, c in enumerate(coords):
          row_sums[i][c] += 1
        col_sum[z_idx] += 1

        if idx % dataInterval == 0  or idx == len(mainDF)-1:

          if len(lhs) == 1:
            u = g1_ub_qp_2D(matrix, row_sums[0], col_sum) / norm
            l = g1_lb_ignoreSum_2D(matrix, row_sums[0], col_sum) / norm

          elif len(lhs) == 2:
            u = g1_bounds_qp_3D(matrix, row_sums[0], row_sums[1], col_sum, False) / norm
            l = g1_bounds_qp_3D(matrix, row_sums[0], row_sums[1], col_sum, True) / norm

          elif len(lhs) == 3:
            u = g1_bounds_qp_4D(matrix, row_sums[0], row_sums[1], row_sums[2], col_sum, False) / norm
            l = g1_bounds_qp_4D(matrix, row_sums[0], row_sums[1], row_sums[2], col_sum, True) / norm

          else:
              continue

          u_final = min(u_final, u)
          l_final = max(l_final, l)

          mainUBDict.setdefault(idx, {})[A_name] = u_final
          mainLBDict.setdefault(idx, {})[A_name] = l_final
          mainLPDict.setdefault(idx, {})[A_name] = (u_final + l_final) / 2

      # pruning
      if l_final > e:
        if len(lhs) == 1:
            banned_attributes.update(lhs)
        continue
      elif u_final <= e:
        attrPrunedLP.append(lhs)
        nextC.append(lhs)
      else:
        nextC.append(lhs)

    # expand eligible lattice after pruning
    currentC = []
    for i in range(len(nextC)):
      for j in range(i + 1, len(nextC)):
        merged = sorted(set(nextC[i] + nextC[j]))
        if len(merged) > 3:
          continue
        if any(attr in banned_attributes for attr in merged):
          continue
        if len(merged) == len(nextC[i]) + 1:
          key = tuple(merged)
          if key not in seen:
            seen.add(key)
            currentC.append(merged)

  return attrPrunedLP, mainLPDict, mainUBDict, mainLBDict

**IPF - Informed Prior**

In [ ]:
!pip install ipfn
import numpy as np
import pandas as pd
from ipfn import ipfn

def calError_2D(matrix):
  error = 0
  for row in matrix:
    pairs = itertools.combinations(row, 2)
    for pair in pairs:
      error += pair[0] * pair[1]
  return error

def calError_3D(matrix):
  error = 0
  for matrix_2d in matrix:
    for row in matrix_2d:
      pairs = itertools.combinations(row, 2)
      for pair in pairs:
          error += pair[0] * pair[1]
  return error

def calError_4D(matrix):
  error = 0
  for matrix_3d in matrix:
    for matrix_2d in matrix_3d:
        for row in matrix_2d:
            pairs = itertools.combinations(row, 2)
            for pair in pairs:
                error += pair[0] * pair[1]
  return error

def calculateIPF(file_name, dataInterval, stopAtIndex, e):

  mainIPFDict = {}
  attrPrunedIPF = []
  banned_attributes = set()

  mainDF = pd.read_csv(file_name, header=0)
  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows - 1)
  A_name_list = mainDF.columns[:-1].tolist()

  currentC = [[a] for a in A_name_list]

  while currentC:

    nextC = []
    seen = set()

    for lhs in currentC:

      if len(lhs) > 3:
        continue  # upto l=3
      if any(attr in banned_attributes for attr in lhs):
        continue  # prune early if attr banned

      A_name = ''.join(lhs)
      Z_name = "y"
      df = mainDF[lhs + [Z_name]]

      # mapping attr values and constructing matrix, rowsum, colsum
      attr_maps = [{val: i for i, val in enumerate(df[col].unique())} for col in lhs]
      z_values = df[Z_name].unique().tolist()
      z_map = {val: i for i, val in enumerate(z_values)}
      shape = [len(m) for m in attr_maps] + [len(z_values)]

      matrix = np.zeros(shape, dtype=int)
      col_sum = [0] * len(z_values)
      row_sums = [[0] * s for s in shape[:-1]]

      # Read rows one by one
      for idx, row in enumerate(df.itertuples(index=False)):

        if idx > stopAtIndex:
          break

        coords = [attr_maps[i][row[i]] for i in range(len(lhs))]
        z_idx = z_map[row[-1]]
        matrix[tuple(coords + [z_idx])] += 1
        for i, c in enumerate(coords):
          row_sums[i][c] += 1
        col_sum[z_idx] += 1

        if idx % dataInterval == 0  or idx == len(mainDF)-1:

          if len(lhs) == 1:
            dimensions_2D = [[0], [1]]
            constraints_2D = [row_sums[0], col_sum]
            ipfMatrix_2D = ipfn.ipfn(np.array(matrix), constraints_2D, dimensions_2D)
            calc_matrix_ipf = np.round(ipfMatrix_2D.iteration()).astype(int)
            error = calError_2D(calc_matrix_ipf)

          elif len(lhs) == 2:
            dimensions_3D = [[0], [1], [2]]
            constraints_3D = [row_sums[0], row_sums[1], col_sum]
            ipfMatrix_3D = ipfn.ipfn(np.array(matrix), constraints_3D, dimensions_3D)
            matrix_2lhs_ipf = np.round(ipfMatrix_3D.iteration()).astype(int)
            error = calError_3D(matrix_2lhs_ipf)

          elif len(lhs) == 3:
            dimensions_4D = [[0], [1], [2], [3]]
            constraints_4D = [row_sums[0], row_sums[1], row_sums[2], col_sum]
            ipfMatrix_4D = ipfn.ipfn(np.array(matrix), constraints_4D, dimensions_4D)
            matrix_3lhs_ipf = np.round(ipfMatrix_4D.iteration()).astype(int)
            error = calError_4D(matrix_3lhs_ipf)

          else:
              continue

          normError = error/norm
          mainIPFDict.setdefault(idx, {})[A_name] = normError

      # pruning
      if normError > e:
        if len(lhs) == 1:
            banned_attributes.update(lhs)
        continue
      elif normError <= e:
        attrPrunedIPF.append(lhs)
        nextC.append(lhs)
      else:
        nextC.append(lhs)

    # expand eligible lattice after pruning
    currentC = []
    for i in range(len(nextC)):
      for j in range(i + 1, len(nextC)):
        merged = sorted(set(nextC[i] + nextC[j]))
        if len(merged) > 3:
          continue
        if any(attr in banned_attributes for attr in merged):
          continue
        if len(merged) == len(nextC[i]) + 1:
          key = tuple(merged)
          if key not in seen:
            seen.add(key)
            currentC.append(merged)

  return attrPrunedIPF, mainIPFDict

**TANE/Greedy - Baseline**

In [10]:
def calBLError_2D(matrix):
  error = 0
  for row in matrix:
    pairs = itertools.combinations(row, 2)
    for pair in pairs:
      error += pair[0] * pair[1]
  return error

def calBLError_3D(matrix):
  error = 0
  for matrix_2d in matrix:
    for row in matrix_2d:
      pairs = itertools.combinations(row, 2)
      for pair in pairs:
        error += pair[0] * pair[1]
  return error

def calBLError_4D(matrix):
  error = 0
  for matrix_3d in matrix:
    for matrix_2d in matrix_3d:
      for row in matrix_2d:
        pairs = itertools.combinations(row, 2)
        for pair in pairs:
          error += pair[0] * pair[1]
  return error

def calculateBaseline(file_name, dataInterval, stopAtIndex, e):

  mainBLDict = {}
  attrPrunedBL = []
  banned_attributes = set()

  mainDF = pd.read_csv(file_name, header=0)
  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows - 1)

  A_name_list = mainDF.columns[:-1].tolist()
  currentC = [[a] for a in A_name_list]

  while currentC:

    nextC = []
    seen = set()

    for lhs in currentC:

      if len(lhs) > 3:
        continue
      if any(attr in banned_attributes for attr in lhs):
        continue

      A_name = ''.join(lhs)
      Z_name = "y"
      df = mainDF[lhs + [Z_name]]

      attr_maps = [{val: i for i, val in enumerate(df[col].unique())} for col in lhs]
      z_values = df[Z_name].unique().tolist()
      z_map = {val: i for i, val in enumerate(z_values)}
      shape = [len(m) for m in attr_maps] + [len(z_values)]

      matrix = np.zeros(shape, dtype=int)
      col_sum = [0] * len(z_values)
      row_sums = [[0] * s for s in shape[:-1]]

      for idx, row in enumerate(df.itertuples(index=False)):

        if idx > stopAtIndex:
          break

        coords = [attr_maps[i][row[i]] for i in range(len(lhs))]
        z_idx = z_map[row[-1]]
        matrix[tuple(coords + [z_idx])] += 1
        for i, c in enumerate(coords):
          row_sums[i][c] += 1
        col_sum[z_idx] += 1

        if idx % dataInterval == 0  or idx == len(mainDF)-1:
          if len(lhs) == 1:
            error = calBLError_2D(matrix)
          elif len(lhs) == 2:
            error = calBLError_3D(matrix)
          elif len(lhs) == 3:
            error = calBLError_4D(matrix)
          else:
              continue

          normError = error/norm
          mainBLDict.setdefault(idx, {})[A_name] = normError

      if normError > e:
        if len(lhs) == 1:
            banned_attributes.update(lhs)
        continue
      elif normError <= e:
        attrPrunedBL.append(lhs)
        nextC.append(lhs)
      else:
        nextC.append(lhs)

    currentC = []
    for i in range(len(nextC)):
      for j in range(i + 1, len(nextC)):
        merged = sorted(set(nextC[i] + nextC[j]))
        if len(merged) > 3:
          continue
        if any(attr in banned_attributes for attr in merged):
          continue
        if len(merged) == len(nextC[i]) + 1:
          key = tuple(merged)
          if key not in seen:
            seen.add(key)
            currentC.append(merged)

  return attrPrunedBL, mainBLDict

**Pyro++ - SOTA**

In [11]:
import re
import os
import pandas as pd
from itertools import combinations

def log_to_dataframe(log_file):

  data = []

  with open(log_file, 'r') as log:
    for line in log:
      if "LHS:" in line and "RHS:" in line and "Error:" in line:
        try:
          lhs = line.split('LHS:')[1].split('RHS:')[0].strip().strip('[]').replace(']', '').replace(',', '').strip()
          rhs = line.split('RHS:')[1].split('Error:')[0].strip().strip('[]').replace(']', '').replace(',', '').strip()
          error = float(line.split('Error:')[1].strip())

          data.append([lhs, rhs, error])
        except IndexError as e:
          print(f"Skipping line due to error: {e}")
          continue

  sorted_data = sorted(data, key=lambda x: x[2])
  df = pd.DataFrame(sorted_data, columns=['LHS', 'RHS', 'Error'])

  return df

def generate_supersets(sets_str):

  supersets = []
  sets = [set(map(int, s)) for s in sets_str]
  unique_elements = set().union(*sets)

  for r in range(1, 4):
      for combo in combinations(unique_elements, r):
          superset = set(combo)
          if any(s.issubset(superset) for s in sets):
              supersets.append(superset)

  supersets = sorted(supersets, key=lambda x: (len(x), sorted(x)))

  supersets_str = [''.join(sorted(map(str, superset))) for superset in supersets]

  return supersets_str

def calculatePyroPrecision(filename, actualTopk, actualG1Dict):

  df_og = pd.read_csv(filename, header = 0)
  pdbxCol = df_og.columns.tolist()
  numCol = df_og.shape[1]

  pyroplusTopk = []
  patkGraph_pyro = []
  ndcgatkGraph_pyro = []
  folder_path = './' + 'logFiles'

  for fn in sorted(os.listdir(folder_path)):

    # note: only pass files upto the percent of data read (60% for our experiments), pass in correct order
    log_file_path = os.path.join(folder_path, fn)
    df_pyro = log_to_dataframe(log_file_path)
    pyroTopK = []

    for index, row in df_pyro.iterrows():
      col1 = row['LHS']
      col2 = row['RHS']
      if col2[6:] == str(numCol):
        col = re.sub(r'[^0-9]', '', col1)
        pyroTopK.append(col)

    pyroplusDictSorted = {}
    pyroplusDictUnsorted = {}

    if pyroTopK:

      pyroplusList = generate_supersets(pyroTopK)
      pyroplusDictUnsorted = {ky: v for ky, v in actualG1Dict.items() if ky in pyroplusList}
      pyroplusDictSorted = dict(sorted(pyroplusDictUnsorted.items(), key=lambda item: item[1]))
      pyroplusTopk = sorted(pyroplusDictSorted, key=lambda ky: pyroplusDictSorted[ky])

      patkGraph_pyro.append(calcPrecision(actualTopk, pyroplusTopk))

    else:
      patkGraph_pyro.append(0.0)

  return patkGraph_pyro, pyroplusTopk

**Score Calculation - Precision**

In [12]:
import numpy as np
import pandas as pd

def excelToDict(file_path):

  df = pd.read_excel(file_path)
  data_dict = {}

  for index, row in df.iterrows():
    index_value = row['Index']
    sub_dict = {column: row[column] for column in df.columns if column != 'Index'}
    data_dict[index_value] = sub_dict

  return data_dict

def calcPrecision(y_true, y_pred):

  y_true_set = set(y_true)
  y_pred_set = set(y_pred)

  true_positives = y_true_set.intersection(y_pred_set)
  precision = len(true_positives) / len(y_pred_set) if len(y_pred_set) > 0 else 0.0

  return precision

def calcRecall(y_true, y_pred):

    y_true_set = set(y_true)
    y_pred_set = set(y_pred)

    true_positives = y_true_set.intersection(y_pred_set)
    recall = len(true_positives) / len(y_true_set) if len(y_true_set) > 0 else 0.0

    return recall

**Generate Results**

In [13]:
import time
import pandas as pd

def calcPrecisionForEpsilonAFD(file_name, dataInterval, rows, e, stopAtIndex, fn, sortedStr):

  prec_lpds, prec_bl, prec_ipf, prec_pyro = [], [], [], []
  rec_lpds, rec_bl, rec_ipf, rec_pyro = [], [], [], []

  # actual top-k
  actualTopkDict, actualDict = calculateActualTopk(file_name, e)
  actualTopk = sorted(actualTopkDict, key=lambda ky: actualTopkDict[ky])

  # approx algorithms
  approxEpslpdsList, lpdsResult, ubDict, lbDict = calculateBoundsLPDS(file_name, dataInterval, stopAtIndex, e)
  approxEpsIpfList, ipfDict = calculateIPF(file_name, dataInterval, stopAtIndex, e)
  approxEpsBlList, blDict = calculateBaseline(file_name, dataInterval, stopAtIndex, e)
  prec_pyro, approxEpsPyroplus = calculatePyroPrecision(file_name, actualTopk, actualDict)

  approxEpslpds = [''.join(attrList) for attrList in approxEpslpdsList]
  approxEpsIpf = [''.join(attrList) for attrList in approxEpsIpfList]
  approxEpsBl = [''.join(attrList) for attrList in approxEpsBlList]

  prec_lpds.append(calcPrecision(actualTopk, approxEpslpds))
  prec_ipf.append(calcPrecision(actualTopk, approxEpsIpf))
  prec_bl.append(calcPrecision(actualTopk, approxEpsBl))

  rec_lpds.append(calcRecall(actualTopk, approxEpslpds))
  rec_ipf.append(calcRecall(actualTopk, approxEpsIpf))
  rec_bl.append(calcRecall(actualTopk, approxEpsBl))

  print('Result after reading', int(stopAtIndex), 'rows:')
  print('AppxLP found', len(approxEpslpds), 'ε-AFDs with Precision =', prec_lpds[-1] ,'; Recall =', rec_lpds[-1])
  print('AppxIPF found', len(approxEpsIpf), 'ε-AFDs with Precision =', prec_ipf[-1] ,'; Recall =', rec_ipf[-1])
  print('TANE found', len(approxEpsBl), 'ε-AFDs with Precision =', prec_bl[-1] ,'; Recall =', rec_bl[-1])
  print('Pyro++ found', len(approxEpsPyroplus), 'ε-AFDs with Precision =', prec_pyro[-1])

  generatePrecisionGraph(prec_lpds, prec_ipf, prec_bl, prec_pyro, rows, dataInterval, e, stopAtIndex, fn, sortedStr)

  return None

**Precision Graph**

In [14]:
import matplotlib.pyplot as plt
import numpy as np

def generatePrecisionGraph(prec_lpds, prec_ipf, prec_bl, prec_pyro, rows, dataInterval, e, stopAtIndex, fn, sortedStr):

  rows_read = list(range(0, round(stopAtIndex) + 1, dataInterval))
  rows_read = [round((x / rows) * 100) for x in rows_read]

  bar_width = 0.15
  index = np.arange(len(rows_read))

  patterns = ['\\', '.', '-', 'x']

  plt.bar(index, prec_lpds, bar_width, label='AppxLP', hatch=patterns[0], facecolor='skyblue', edgecolor='black')
  plt.bar(index + bar_width, prec_ipf, bar_width, label='AppxIPF', hatch=patterns[1], facecolor='khaki', edgecolor='black')
  plt.bar(index + 2 * bar_width, prec_bl, bar_width, label='TANE', hatch=patterns[2], facecolor='darkseagreen', edgecolor='black')
  plt.bar(index + 3 * bar_width, prec_pyro, bar_width, label='Pyro++', hatch=patterns[3], facecolor='lightsalmon', edgecolor='black')

  plt.xlabel('Percent of Data Read', fontsize=16, fontweight='bold')
  plt.ylabel('Precision', fontsize=16, fontweight='bold')

  plt.xticks(index + bar_width * 1.75, [f'{round(r, 0)}' for r in rows_read], fontsize=16)
  plt.yticks(fontsize=16)

  plt.grid(True, linestyle='--', linewidth=0.5)
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.22), ncol=2, prop={'weight': 'bold', 'size': 14})

  plt.savefig('Precision_e='+str(e)+'_'+fn+'_'+sortedStr+'.png', dpi=300, bbox_inches='tight')
  plt.close()

**Dataset Sorting (Key/Non-Key)**

In [15]:
import pandas as pd

def sortDataset(file_name, attrList, attr, sortOrder):

  if attr != '0':
    df = pd.read_csv(file_name)
    df_sorted = df.sort_values(by=attrList, ascending=sortOrder)
    sortedFN = 'Sorted_X'+attr+'_'+file_name
    df_sorted.to_csv(sortedFN, index=False)
  else:
    return file_name

  return sortedFN

**Start Point** (Define parameters and run code)

In [16]:
def getDatasetParam(datasetName):

    dataset_config = {
        'dataset1': {'fn': 'DB Status', 'rows': 10000, 'interval': 1000, 'fileName': 'DBStatus.csv'},
        'dataset2': {'fn': 'Letter', 'rows': 20000, 'interval': 2000, 'fileName': 'Letter.csv'},
        'dataset3': {'fn': 'Student', 'rows': 1000, 'interval': 100, 'fileName': 'Student.csv'},
        'dataset4': {'fn': 'Synthetic1', 'rows': 100000, 'interval': 10000, 'fileName': 'SyntheticData1.csv'},
        'dataset5': {'fn': 'Synthetic2', 'rows': 100000, 'interval': 10000, 'fileName': 'SyntheticData2.csv'},
        'dataset6': {'fn': 'Synthetic3', 'rows': 160000, 'interval': 16000, 'fileName': 'SyntheticData3.csv'},
        'dataset7': {'fn': 'Synthetic4', 'rows': 48000, 'interval': 4800, 'fileName': 'SyntheticData4.csv'}
    }

    if datasetName in dataset_config:
        return dataset_config[datasetName]
    else:
        raise ValueError(f"{datasetName} not found.")

def AFDStartPoint():

  # define dataset paramters
  datasetNum = 'dataset4'
  params = getDatasetParam(datasetNum)
  fn = params['fn']
  rows = params['rows']
  interval = params['interval']
  fileName = params['fileName']

  # define sorting order
  attr = 'y'
  attrList = ['y']
  sortOrder = [True]
  sortedStr = '(Sorted By Key Attribute)'
  data = sortDataset(fileName, attrList, attr, sortOrder)

  # threshold used by pyro
  e = 0.01
  # define limit of records read
  stopAtIndex = 0.6*rows

  # pass the parameters and generate results
  calcPrecisionForEpsilonAFD(data, interval, rows, e, stopAtIndex, fn, sortedStr)

In [ ]:
AFDStartPoint()